In [7]:

import cv2
import numpy as np
import os
from matplotlib import pylab
import pylab as pl
from sklearn.metrics import confusion_matrix, accuracy_score 
# from sklearn.externals import joblib
import joblib

from scipy.cluster.vq import vq    

In [18]:
clf, classes_names, stdSlr, k, voc = joblib.load("bovw2.pkl")

In [19]:
# TODO : Descriptor Types
brisk = cv2.BRISK_create(30)
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(nfeatures=30,scoreType=cv2.ORB_FAST_SCORE)
sift = cv2.xfeatures2d.SIFT_create()

In [20]:
test_path = 'Images/Test'

# window_size = [(32,32),(64,64),(128,128)]
window_size = [(128,128)]

images = [os.path.join(test_path,f) for f in os.listdir(test_path)]

dims = (512,512)

for w in window_size:
    print("Window : ",w)
    for image in images[:1]:
        print("Reading img : ",image)
        img = cv2.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(gray, (512, 512))
        
#         cv2.imshow("Image", gray)
#         cv2.waitKey(0)   
#         cv2.destroyAllWindows()
        
        height, width= resized_image.shape
#         print("Dimensions : ",width,height)
        window_number = 1
        windows = []

        y=0

        des_list = []

        while y<height:
            if(y+w[1]>dims[1]):
                break  
            x=0
            while x<width:
                if (x+w[0])>dims[0]:
                    break
                print(window_number," : ",x,",",y)
                window_number+=1
                imgWindow = resized_image[y:y+w[1], x:x+w[0]]

                kpts, des = brisk.detectAndCompute(imgWindow, None)
                des_list.append((window_number, des))
                # kp, des = orb.detectAndCompute(imgWindow, None)
                # matches = bf.match(des1,des2)
                
                # cv2.imshow("Window Image", imgWindow)
                # cv2.waitKey(0)   
                # cv2.destroyAllWindows()
                
                x = x + w[0]

            y = y+w[1]

        # For One Image, All Windows Scanned!

        # Stack all the descriptors vertically in a numpy array

        # TODO : Apply this Image to Train with BOVW generated histograms!

        print("des_list :\n",len(des_list[0]))

        descriptors = des_list[0][1]
        
        for window_number, descriptor in des_list[1:]:
            # descriptors = np.vstack((descriptors, descriptor))
            np.concatenate((descriptors,np.concatenate(descriptor)[:,None]),axis=1)

        test_features = np.zeros((window_number, k), "float32")

        for i in range(window_number):
            words, distance = vq(des_list[i][1],voc)
            for w in words:
                test_features[i][w] += 1
                

        # Perform Tf-Idf vectorization
        nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
        idf = np.array(np.log((1.0*window_number+1) / (1.0*nbr_occurences + 1)), 'float32')

        test_features = stdSlr.transform(test_features)

        predictions =  [classes_names[i] for i in clf.predict(test_features)]

        print ("prediction ="  + str(predictions))

Window :  (128, 128)
Reading img :  Images/Test/B0046_0012.png
1  :  0 , 0
2  :  128 , 0
3  :  256 , 0
4  :  384 , 0
5  :  0 , 128
6  :  128 , 128
7  :  256 , 128
8  :  384 , 128
9  :  0 , 256
10  :  128 , 256
11  :  256 , 256
12  :  384 , 256
13  :  0 , 384
14  :  128 , 384
15  :  256 , 384
16  :  384 , 384
des_list :
 2


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 64